# <span style="color:gold">**Agregando interactividad en Python**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este tutorial, le daremos un vistazo a <span style="color:gold">ipywidgets</span> y sus herramientas interactivas para el análisis exploratorio de datos.

Este Notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.
***

## **1. Interactividad dentro de un notebook de Jupyter**

Un <span style="color:gold">notebook de Jupyter</span> es una herramienta muy poderosa para el análisis exploratorio de datos.\
Gracias a Jupyter, podemos cargar, procesar y visualizar toda nuestra información. También podemos desarrollar software y generar nuevos resultados.

Los científicos de datos utilizan Jupyter para documentar su trabajo, explorar y experimentar con nuevos algoritmos y flujos de trabajo.

Es en esta actividad que el uso de <span style="color:gold">herramientas interactivas</span> es tan importante.\
Los <span style="color:gold">widgets de Jupyter</span> nos permiten visualizar resultados o crear mini-aplicaciones web que faciliten la exploración del contenido y la interacción con los datos.

***

## **2. Interacción con valores numéricos**
Empezamos importando la librería de widgets de Jupyter:

In [ ]:
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Podemos utilizar los siguientes widgets para interactuar con valores numéricos:
### **IntSlider**

In [ ]:
widgets.IntSlider(min=0, max=10, step=1, description="Integer:", value=3)

### **FloatSlider**

In [ ]:
widgets.FloatSlider(min=0., max=10., step=0.5, description="Float:")

### **IntRangeSlider**

In [ ]:
widgets.IntRangeSlider(value=[5, 7], min=0, max=10, step=1, description="Intervalo:")

### **FloatRangeSlider**

In [ ]:
widgets.FloatRangeSlider(value=[5, 7.5], min=0, max=10.0, step=0.1, description="Intervalo:", readout_format=".1f")

### **FloatLogSlider**

In [ ]:
widgets.FloatLogSlider(value=10, base=10, min=-3, max=3, step=0.2, description="Log")

## **3. Interacción con valores categóricos**
Podemos utilizar los siguientes widgets para interactuar con valores categóricos:
### **CheckBox**

In [ ]:
widgets.Checkbox(value=False, description="Filtrar información", disabled=False)

### **Dropdown**

In [ ]:
widgets.Dropdown(options=["Andesita", "Dacita", "Basalto"], value="Basalto", description="Litología:", disabled=False)

In [ ]:
widgets.Dropdown(options=[("Andesita", 1), ("Dacita", 2), ("Basalto", 3)], value=2, description="Litología:")

## **4. Generación de resultados interactivos**
Si usamos la función `interactive_output`, podemos enlazar un widget con cualquier función:

In [ ]:
slider = widgets.FloatSlider(min=0., max=10., step=0.5, description="Número:")

# Función a enlazar
def cubo(n):
    print(n**3)

# Enlace entre el widget y la función
dashboard = widgets.interactive_output(cubo, {"n": slider})
dashboard.clear_output(wait=True)

# Muestra los widgets
display(slider, dashboard)

También podemos usar la función `interact`:

In [ ]:
def three(x, y, z):
    return (x, y, z)

widgets.interact(three, x=(0, 10, 1), y=True, z=["Andesita", "Dacita", "Basalto"]);

### **4.1. Gráficos interactivos usando Matplotlib**

Usaremos una función llamada `dispersion_lineal` para generar un gráfico en Matplotlib que represente una ecuación lineal de tipo `ax + b` y que además sea interactiva a través de la función `interact`, tomando como parámetros de variación los valores de `a`, `b` y una opción adicional que grafique una regresión lineal de los puntos.

In [ ]:
# Input
x = np.random.randn(1000)
ruido = np.random.randn(1000)
z = np.random.randn(1000)

# Mapa de colores
cmap = plt.cm.inferno

def dispersion_lineal(a=3, b=1, lin_reg=False):
    # Combinación lineal
    y = a*x + b + ruido
    
    # Figura principal
    fig, ax = plt.subplots(figsize=(6, 6))

    # Diagrama de dispersión
    im = ax.scatter(x, y, c=z, cmap=cmap, s=50, edgecolor="black", alpha=0.7)
    
    # Límites
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)    
    
    # Título y grilla
    ax.set_title(f"Y = {a}X + {b}", fontsize=20)
    ax.grid(color="black", linewidth=0.5, alpha=0.5)
    
    # Regresión lineal
    if lin_reg:
        b, a = np.polyfit(x, y, deg=1)
        line = np.linspace(-5, 5, num=20)
        ax.plot(line, a + b*line, color="blue", lw=2)    
    
    plt.tight_layout()
    
widgets.interact(dispersion_lineal, a=(-10, 10, 1), b=(-5, 5, 1), lin_reg=False);

### **4.2. Tabla interactiva usando Pandas**

En este ejemplo, usaremos el archivo `rocas.csv` ubicado en la carpeta `files` para visualizar un DataFrame de manera interactiva.

In [ ]:
dataframe = pd.read_csv("files/rocas.csv")

# Opción para que el DataFrame redondee el formato de visualización de los datos numéricos 
pd.set_option("display.float_format", lambda x: f"{x:.4f}")

In [ ]:
# Objeto que representa la tabla de información
class Data:
    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe
        # La información debe ser limpiada antes de usar la función
        num_cols = []
        cat_cols = []
        for col in dataframe.columns:
            if dataframe[col].dtype in [int, float]:
                num_cols.append(col)
            else:
                cat_cols.append(col)
        self.num_cols = num_cols
        self.cat_cols = cat_cols

        
# Objeto que representa el widget deslizador de valores en una columna    
class ColumnSlider:
    def __init__(self, dataframe, name):
        self.name = name
        self.col_min = int(dataframe[name].min())
        self.col_max = int(dataframe[name].max())
        self.widget = widgets.SelectionRangeSlider(options=np.arange(self.col_min, self.col_max + 1, 1),
                                                   index=(0, 1),
                                                   value=(self.col_min, self.col_max),
                                                   description=f"{self.name}",
                                                   layout={'width': '400px'})


# Objeto que representa la interfaz en la cual visualizaremos la tabla y los deslizadores        
class DashBoard:
    def __init__(self, dataframe):
        self.data = Data(dataframe)
        self.df = self.data.dataframe
        self.sliders = [ColumnSlider(dataframe, name) for name in self.data.num_cols]
        
        def update_dataframe(**kwargs):
            df = self.data.dataframe
            for col, slider in kwargs.items():
                start = slider[0]
                end = slider[1]
                df = df[(df[col] >= start) & (df[col] <= end)]
            self.df = df
            display(df)
        
        dict_sliders = {f"{slider.widget.description}": slider.widget for slider in self.sliders}
        self.output = widgets.interactive_output(update_dataframe, dict_sliders)
        self.output.clear_output(wait = True)
        
        file_widget = widgets.Text(placeholder="Nombre del archivo", continuous_update=False)
        save_button = widgets.Button(description="Guardar archivo csv", button_style="", tooltip="Guardar archivo",
                                     icon="floppy-o", style={"button_color": "green"})
        
        def save_click(b):
            filename = file_widget.value
            if filename in ["rocas", "", "Nombre no válido"]:
                file_widget.value = "Nombre no válido"
            else:
                with self.output:
                    self.df.to_csv(f"{filename}.csv", index=False)
                
        save_button.on_click(save_click)
        
        file_container = widgets.VBox([file_widget, save_button])
        slider_container = widgets.VBox([slider.widget for slider in self.sliders])
        left = widgets.VBox([slider_container, file_container])
        right = widgets.VBox([self.output])
        display(widgets.HBox([left, right]))

        

In [ ]:
DashBoard(dataframe);

***